<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/ALE_Framework_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("This is a test")

This is a test
